In [1]:
import mesa
import seaborn as sns
import numpy as np
import random
import pandas as pd
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import statistics

In [ ]:
# Development of an agent-based simulation model in combination with reinforcement learning in Python using Mesa library



# - At the beginning of an episode, 10 plants (as agents) are planted

# - Plants must grow for 10 days (steps) before they can be harvested.

# - Each plant has a 10% chance of dying every day.

# - A new (fresh) plant can be bought every day (cost $10) to be planted

# - The aim is to harvest 10 plants that each grew for 10 days. When the goal is reached, there is a reward of $20 per plant harvested and the episode ends

# - Each day of the episode costs $5

# - Reinforcement learning is now used to find a strategy when to plant new trees that minimizes total costs.


# UPDATES

# Just 2 things:



# 1) May I ask you to include the possibility to run the simulation environment without the RL optimization. Following variables should be possible to be defined:
# - a daily probability for the decision variable to buy or not to buy.
# - number of episodes, the simulation will run
# At the end of each episode, I'd like to see a table with the days (horizontal) and the plants (vertical) showing the different states of each plant on each day.
# At the end of the entire simulation, I'd like to see a table showing important values of each episode, such as:
# - the total costs
# - the number of days needed



# 2) May I also ask you to precisely comment the code, in order I can follow and understand it?

In [16]:
class PlantAgent(mesa.Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

        self.alive = True
        self.age = 0
        self.harvested = False

    def step(self):
        if self.alive and self.age != 10:
            self.age += 1

        if self.age == 10:
            self.harvested = True


        if not self.alive or self.harvested:
            return


        death_outcome = random.choices([False, True], weights=[0.1, 0.9], k=1)[0]
        self.alive = death_outcome


In [15]:
class PlantModel(mesa.Model):

    def __init__(self, N):
        super().__init__()

        self.num_agents = N
        self.number_of_days = 0
        self.number_of_plants_harvested = 0

        self.schedule = mesa.time.RandomActivation(self)

        for i in range(self.num_agents):
            a = PlantAgent(i, self)
            self.schedule.add(a)

    def get_state(self):
        agents = [a for a in self.schedule.agents]
        state = []

        number_of_plants_harvested = 0

        for a in agents:

            if a.alive and not a.harvested:
                state.append(a.age)

            if a.harvested:
                number_of_plants_harvested += 1


        return number_of_plants_harvested, tuple(sorted(state))

    def check_terminated(self, observation):
        return list(map(lambda x: x[1] == 10, observation)).count(True)

    def step(self, action):

        self.number_of_days += 1
        terminated = 0
        truncated = False
        reward = -5

        if action == 1:
            self.num_agents += 1
            a = PlantAgent(self.num_agents - 1, self)
            self.schedule.add(a)

            reward -= 10

        self.schedule.step()

        observation = self.get_state()


        difference = observation[0] - self.number_of_plants_harvested
        reward += difference * 20

        self.number_of_plants_harvested = observation[0]

        if self.number_of_days == 500:
            truncated = True

        if self.number_of_plants_harvested == 10:
            terminated = 1




        return observation, reward, terminated, truncated

In [17]:
model = PlantModel(10)

In [18]:
model.step(0)

((0, (1, 1, 1, 1, 1, 1, 1, 1)), -5, 0, False)

In [20]:
agents = [a for a in model.schedule.agents]

In [ ]:
sorted(agents, key=)

In [25]:
# id = []
# alive = []
# age = []
# harvested = []

agent_data = []

for a in agents:

    agent_data.append((a.unique_id, a.alive, a.age, a.harvested))

    # id.append(a.unique_id)

    # print(f"{a.unique_id}  {a.age}")

In [28]:
sorted_agent_data = sorted(agent_data, key=lambda x: x[0])

In [29]:
sorted_agent_data

[(0, True, 1, False),
 (1, True, 1, False),
 (2, True, 1, False),
 (3, False, 1, False),
 (4, True, 1, False),
 (5, True, 1, False),
 (6, True, 1, False),
 (7, True, 1, False),
 (8, False, 1, False),
 (9, True, 1, False)]

In [34]:
def create_plant_data_string(plant_data):
    plant_data_string = ""

    if plant_data[1] == True:
        plant_data_string += "Alive"
    else:
        plant_data_string += "Dead"

    plant_data_string += "   Age:"
    plant_data_string += str(plant_data[2])
    plant_data_string += "   "

    if plant_data[3] == True:
        plant_data_string += "Harvested"
    else:
        plant_data_string += "Not Harvested"

    return plant_data_string

In [35]:
create_plant_data_string(sorted_agent_data[0])

'Alive   Age:1   Not Harvested'

Thera are a few things to consider

Terminal State: This would be achieved when we have 10 plants harvested.  
Truncation: The environment would truncate in 500 steps.  
Reward: Positive for harvesting and negative for buying a plant and on every step.  
Observation: This would be the current state. The state would be the number of plants harvested and the age of each plant.  

In [4]:
# DO NOT RUN THIS - THIS is JUST TO ESTIMATE THE STATE SPACE

def get_state_space(number_of_iterations = 100000):

    state_space = set()

    truncated = 0
    model = PlantModel(10)
    initial_state = model.get_state()
    state_space.add(initial_state)


    for _ in tqdm(range(number_of_iterations)):

        model = PlantModel(10)
        observation = []
        terminated = 0



        while not terminated and not truncated:

            action = random.randint(0, 1)
            observation, _, terminated, truncated = model.step(action)
            state_space.add(observation)

    state_space = list(state_space)
    state_space = sorted(state_space, key=lambda x: (x[0], x[1]))

    return state_space

In [18]:
# DO NOT RUN
# state_space = get_state_space(10)

In [19]:
# DO NOT RUN THIS - THIS IS JUST TO STORE THE STATE SPACE


# with open("state_space.pkl", "wb") as file:
#     pickle.dump(state_space, file)

In [5]:
with open("state_space.pkl", "rb") as file:
    state_space = pickle.load(file)

In [10]:
def initialize_Q_Table(state_space):
    Q_Table = {}

    for s in state_space:
        Q_Table[s] = [0, 0]

    return Q_Table

In [11]:
Q_Table = initialize_Q_Table(state_space)

In [13]:
def generate_trajectory(policy):

    trajectory = []

    state = (0, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
    terminated = 0
    truncated = 0
    next_state = (0, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

    model = PlantModel(10)

    while not terminated and not truncated:
        state = next_state
        action = policy[state]

        observation, reward, terminated, truncated = model.step(action)

        next_state = observation
        experience_tuple = (state, action, reward, next_state)

        trajectory.append(experience_tuple)

    return trajectory

In [14]:
class QLearning:
    def __init__(self, Q_Table, gamma=1, number_of_iterations=100000):
        self.number_of_iterations = number_of_iterations

        self.Q = Q_Table

        self.ε = self.get_parameters_exponential_decay(decay_rate=0.999995)
        self.α = self.get_parameters_exponential_decay(decay_rate=0.999995)

        self.trajectories = [[]]

    def epsilon_greedy_exponential(self, iteration, s):
        ε = self.ε[iteration]

        a = 0

        if s not in self.Q:
            self.Q[s] = [0, 0]

        if np.random.random() > ε:
            a = np.argmax(self.Q[s])
        else:
            a = np.random.randint(len(self.Q[s]))

        return a

    def get_parameters_exponential_decay(
        self, initial_value=1, min_value=0.01, decay_rate=0.99
    ):
        num_points = self.number_of_iterations

        exponential_decay_parameters = initial_value * (
            decay_rate ** np.arange(num_points)
        )
        exponential_decay_parameters = np.where(
            exponential_decay_parameters < min_value,
            min_value,
            exponential_decay_parameters,
        )

        return exponential_decay_parameters

    def do_one_qlearning_iteration(self, iteration, γ=0.99):

        s = (0, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
        terminated = 0
        truncated = 0
        s_prime = (0, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

        trajectory = []

        model = PlantModel(10)
        while not terminated and not truncated:
            a = self.epsilon_greedy_exponential(iteration, s)
            s_prime, R, terminated, truncated= model.step(a)


            if s_prime not in self.Q:
                self.Q[s_prime] = [0, 0]




            self.Q[s][a] = self.Q[s][a] + self.α[iteration] * (
                R + γ * max(self.Q[s_prime]) - self.Q[s][a]
            )

            trajectory.append((s, a, R, s_prime))

            s = s_prime

        return trajectory

    def do_qlearning(self):
        for i in tqdm(range(self.number_of_iterations)):
            trajectory = self.do_one_qlearning_iteration(i)
            self.trajectories.append(trajectory)


In [205]:
QL = QLearning(new_Q_Table, number_of_iterations=1000000)

In [206]:
QL.do_qlearning()

100%|██████████| 1000000/1000000 [1:45:58<00:00, 157.28it/s] 


In [ ]:
Q_Table = QL.Q

In [209]:
# DO NOT RUN THIS CELL
# with open("Q_Table.pkl", "wb") as file:
#     pickle.dump(Q_Table, file)

In [10]:
with open("Q_Table.pkl", "rb") as file:
    loaded_Q_Table = pickle.load(file)

In [11]:
def get_policy(Q_Table):

    policy = {}

    for state in Q_Table.keys():
        optimal_action = np.argmax(loaded_Q_Table[state])
        policy[state] = optimal_action

    return policy

In [12]:
policy = get_policy(loaded_Q_Table)

In [24]:
# # DO NOT RUN THIS CELL
# with open("Policy.pkl", "wb") as file:
#     pickle.dump(policy, file)

In [25]:
with open("Policy.pkl", "rb") as file:
    policy = pickle.load(file)

In [5]:
def get_sum_of_reward_from_multiple_trajectories(policy, number_of_trajectories=1000):

    sum_of_rewards = []

    for _ in tqdm(range(number_of_trajectories)):

        trajectory = generate_trajectory(policy)

        rewards = list(map(lambda x: x[2], trajectory))
        sum_of_rewards.append(sum(rewards))

    return sum_of_rewards

In [21]:
def get_statistics_of_total_reward(policy):
    sum_of_rewards = get_sum_of_reward_from_multiple_trajectories(policy, 1000)
    mean_reward = statistics.mean(sum_of_rewards)
    std_dev = statistics.stdev(sum_of_rewards)

    return mean_reward, std_dev

In [23]:
get_statistics_of_total_reward(policy)

100%|██████████| 1000/1000 [00:02<00:00, 447.43it/s]


(-109.715, 107.80823897732209)